# FERM experiment examples with Adult and Drug datasets

In [ ]:
import numpy as np
import time
from collections import namedtuple

from datasets import *
from learning_algorithms import *
from fairness_metrics import fairness_scores

## Linear FERM on drug dataset

In [ ]:
# Drug Consumption, Arrhythmia and German Credit datasets need to be loaded in this way. 
data_full = drug()
data = data_full[0]
labels = data_full[1]
m, n = data.shape
train_data, train_labels, test_data, test_labels = split(data, labels, m)
train = namedtuple('_', 'data, labels')(train_data, train_labels)
g = 5 # ethnicity (white, ethnic minority)

In [ ]:
runs = 10

acc = []
eo = []
eod = []
dp = []
te = []
S = []

for r in range(runs): # runs not required for Adult as Adult comes with train/test splits
    
    # new data shuffle and split each run
    data_full = drug()
    data = data_full[0]
    labels = data_full[1]
    m, n = data.shape
    train_data, train_labels, test_data, test_labels = split(data, labels, m)
    train = namedtuple('_', 'data, labels')(train_data, train_labels)
    g = 5
    
    # train model. loss type options: HINGE, MSE, CE. fairness_transform options: DEO, DDP
    LA = LinearAlg(loss_type='MSE', fairness_transform='DEO')
    LA.execute(train, g)  
    
    # get predictions
    train_preds = LA.predict(train_data)
    test_preds = LA.predict(test_data)
    
    # get accuracy
    acc_te = accuracy_score(test_labels, test_preds)
    acc_tr = accuracy_score(train_labels, train_preds)
    acc.append(acc_te)
    
    # get fairness scores
    tr_s = fairness_scores(train_data, train_labels, train_preds, g)
    train_scores = tr_s.get_scores2()

    te_s = fairness_scores(test_data, test_labels, test_preds, g)
    test_scores = te_s.get_scores2()

    scores = [acc_te, test_scores[0], test_scores[1],  test_scores[2], test_scores[3]]
    
    S.append(scores)
    eo.append(test_scores[0])
    eod.append(test_scores[1])
    dp.append(test_scores[2])
    te.append(test_scores[3])
    
S = np.array(S)

In [ ]:
# get mean and standard deviations of scores
for i in range(len(S[0,:])):
    avg = np.mean(S[:,i])
    sd = np.std(S[:,i])
    print('mean=', avg, 'sd=',sd)

## Non-linear FERM on Adult dataset

In [ ]:
train, test = adult()
train_data = train[0]
train_labels = train[1]
test_data = test[0]
test_labels = test[1]

g = 9 # female or male

In [ ]:
# train model
gamma = 1
C_val = 0.1
LA = Non_LinearAlg(algorithm='FERM')
LA.execute(train, g, gamma=gamma, C=C_val)

# get predictions and accuracy
train_preds, acc_tr = LA.predict(train_data, train_labels)
test_preds, acc_te = LA.predict(test_data, test_labels)

# get fairness scores
tr_s = fairness_scores(train_data, train_labels, train_preds, g)
train_scores = tr_s.get_scores()

te_s = fairness_scores(test_data, test_labels, test_preds, g)
test_scores = te_s.get_scores()

scores = [acc_tr, train_scores[0], train_scores[1],  train_scores[2], train_scores[3], 
            acc_te, test_scores[0], test_scores[1],  test_scores[2], test_scores[3]]

print("TRAINING. Accuracy: {}, Equal opportunity: {}, Equalized odds: {}, Demographic parity: {}, Treatment equality: {}".format(scores[0], scores[1], scores[2], scores[3], scores[4]))
print("TESTING. Accuracy: {}, Equal opportunity: {}, Equalized odds: {}, Demographic parity: {}, Treatment equality: {}".format(scores[5], scores[6], scores[7], scores[8], scores[9]))